1.輸入參數


In [ ]:
import requests
import csv

media_server_ip = input("請輸入Ant Media Server IP (e.g. 61.222.163.86):")
media_server_port = int(input("請輸入 Port (預設 5080):") or 5080)
username = input("請輸入username (e.g. admin)")
password = input("請輸入password(e.g. ez008131)")
zone = input("請輸入機台區域 (e.g. A/B/C/D):").strip().upper()
subnet = int(input("請輸入網段(e.g. 11/12/13/14):").strip())
start_id = int(input("請輸入起始 ID (包含):"))
end_id = int(input("請輸入結束 ID (包含):"))

2.定義(新增/匯出/刪除)函式


In [ ]:
import requests
import csv
import hashlib
import hmac

def generate_stream_id(name: str, secret_key: str = "ams") -> str:
    h = hmac.new(secret_key.encode(), name.encode(), hashlib.md5)
    return f"{name}_{h.hexdigest()}"

def generate_hash_from_name(name: str, secret_key: str = "ams") -> str:
    h = hmac.new(secret_key.encode(), name.encode(), hashlib.md5)
    return h.hexdigest()
#ip變數規則
def ip_location(subnet, id):
    if subnet == 11:
        return id
    elif subnet == 12:
        return id -200
    elif subnet in (14, 15):
        return id - 300
    elif subnet == 13:
        if id > 400:
            return id - 400
        elif id > 300:
            return id - 300
        else:
            return id
    else:
            raise ValueError


#Camera-Create 批量建立攝影機
def do_create():
    results = []
    fail_count = 0
    url = f"http://{media_server_ip}:{media_server_port}/WebRTCAppEE/rest/v2/broadcasts/create?autoStart=true"

    is_backup_input = input("這批攝影機是否為備源？輸入 yes 則會自動加 -1：").strip().lower()
    is_backup_batch = is_backup_input == "yes"
    
    for id in range(start_id, end_id + 1):
        suffix  = ip_location(subnet, id)
        ip_addr = f"192.168.{subnet}.{suffix}"
        
        base_name = f"{zone}{id if id > 100 else f'0{id}'}"
        name = f"{base_name}-1" if is_backup_batch else base_name
        stream_id = f"{name}_{generate_hash_from_name(base_name)}"
        
        data = {
            "streamId": stream_id,
            "hlsViewerCount": 0,
            "dashViewerCount": 0,
            "webRTCViewerCount": 0,
            "rtmpViewerCount": 0,
            "mp4Enabled": 0,
            "playlistLoopEnabled": True,
            "autoStartStopEnabled": False,
            "plannedStartDate": 0,
            "playListItemList": [],
            "ipAddr": ip_addr,
            "name": name,
            "username": username,
            "password": password,
            "type": "ipCamera",
        }
        print(data)

        response = requests.post(url, json=data, timeout= 10)

        if response.status_code == 200:
            results.append({'id': id, 'name': name})
        else:
            fail_count += 1
            print(f"❌ {name} 建立失敗:HTTP {response.status_code} → {response.text}")

    success_count = len(results)
    total_count = end_id - start_id +1
    print(f"建立完畢: 共{total_count}，成功 {success_count} 筆，失敗{fail_count}筆。")
    # ✅ 建立成功後寫入 CSV

    with open("created_streams.csv", mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file, quoting=csv.QUOTE_MINIMAL)
        writer.writerow(["Name", "Stream ID"])
        for r in results:
            base_name = r["name"]
            stream_id = f"{base_name}_{generate_hash_from_name(base_name.replace('-1',''))}"
            writer.writerow([base_name, stream_id])

print("✅ 已匯出 created_streams.csv")
def fetch_all_broadcasts():
    all_data = []
    offset = 0
    batch_size = 1000
    while True:
        url = (
            f"http://{media_server_ip}:{media_server_port}/WebRTCAppEE/rest/v2/broadcasts/list/{offset}/{batch_size}"
        )
        r = requests.get(url, timeout=20)
        r.raise_for_status()
        batch = r.json()
        if not batch:
            break
        all_data.extend(batch)
        if len(batch) < batch_size:
            break  # 最後一批
        offset += batch_size
    return all_data
def do_export():
    data = fetch_all_broadcasts()

    # 🔽 新增使用者輸入：zone 前綴（如 A、B、C）
    zone_prefix = input("請輸入要匯出的區域（如 A/B/C）：").strip().upper()

    # ✅ 篩選條件：名稱以指定字母開頭，且不是備源（不以 -1 結尾）
    filtered = [i for i in data if i["name"].startswith(zone_prefix) and not i["name"].endswith("-1")]

    def name_sort_key(x):
        name = x["name"]
        digits = ''.join(filter(str.isdigit, name[1:]))
        return int(digits) if digits.isdigit() else float("inf")

    filtered.sort(key=name_sort_key)

    with open("stream_id_sheet.csv", mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file, quoting=csv.QUOTE_MINIMAL)
        writer.writerow(["Name", "Stream ID", "Camera IP"])
        for i in filtered:
            writer.writerow([i["name"], i["streamId"], i.get("ipAddr", "")])

    print(f"✅ 區域 {zone_prefix} 主源匯出完成，共 {len(filtered)} 筆 → stream_id_sheet.csv")
def do_delete():
    list_url = (
        f"http://{media_server_ip}:{media_server_port}"
        "/WebRTCAppEE/rest/v2/broadcasts/list/0/1000"
        "?sort_by=name&order_by=asc"
)
    r = requests.get(list_url, timeout=10)
    r.raise_for_status()
    data = r.json()

    prefix = input("請輸入所要刪除的機台區域 (e.g. A/B/C/D):").strip().upper()
    delete = [i["streamId"] for i in data if i["name"].startswith(prefix)]
    if not delete:
        print(f"⚠️ 沒有找到任何以「{zone}」開頭的機台。")
        return
    
    print(f"\n 你選擇刪除區域：{prefix} 區，共 {len(delete)} 台機台：")
    for id in delete:
        print("",id)
        
    confirm = input("\n確認要刪除以上所有機台嗎？輸入 yes 執行，其它任意鍵取消：").strip().lower()
    if confirm != "yes":
        print("❌ 已取消刪除操作。")
        return
    
    base = list_url.rsplit("/list",1)[0]
    for id in delete:
        url_del = f"{list_url.rsplit('/list',1)[0]}/{id}" #v2 broadcasts/id
        rd = requests.delete(url_del, timeout=10)
        if r.status_code in (200, 204):
            print("刪除成功")
        else:
            print(f"❌ 刪除失敗：{id} → HTTP {r.status_code},{r.text}")
        
    print(f"\n🔚 完成，共刪除 {len(delete)} 支「{zone}」開頭的機台。")

功能預覽層(不執行)


In [ ]:
total = end_id - start_id + 1

print("=== 功能預覽 ===\n")

# 預覽 1) 建立
print("[1] 建立攝影機")
print(f"     → 區域：{zone}")
print(f"     → 網段：192.168.{subnet}.X")
print(f"     → ID 範圍：{start_id} ～ {end_id}，共 {total} 台")
print("     → 會逐台呼叫 Create API 建立\n")

# 預覽 2) 匯出
print("[2] 匯出 Stream List")
print(f"     → 呼叫出192.168.{subnet}.{ip_location}的所有機台資料")
print("     → 產出分別for sql/sheet的檔案：")
print("         • stream_id_sql.csv")
print("         • stream_id_sheet.csv\n")

# 預覽 3) 刪除
print("[3] 刪除攝影機 ※此動作將刪除所指定的區域所有攝影機")
print("     → 讓你輸入要刪除的區域前綴（例如 A/B/C/D/E）")
print("     → 會過濾該前綴並列出要刪除的 StreamID，再二次確認\n")

print("請執行下一格，輸入 1/2/3 選擇要執行的功能。")

選擇(新增/匯出/刪除)操作


In [ ]:
print("請選擇功能：1) 建立  2) 匯出  3) 刪除")
choice = input("輸入 1/2/3：").strip()
if choice == "1":
    do_create()
elif choice == "2":
    do_export()
elif choice == "3":
    do_delete()
else:
    print("無效選項，請重新執行本 Cell。")